# Fastai Experimentation

This notebook is intended to be used after running the google image scraper in this repo. To get started follow the README.md in this repo, then come back after all the csv files have been generated, labeled, and saved in the data_csv directory.

In [1]:
from fastai.vision import *
import os
path = Path('../../data_csv')

### Define Experiment
Running this cell will save the images and any saved models in a directory with the experiments name.

In [2]:
experiment = 'trees_and_leaves_v2'

In [3]:
dest_path = Path('../../data')
img_path = dest_path/experiment

## Download images per class
The google scraper will save the csv files labeled by the classes you want to train. These csv files will be nested in a directory with the labeled classname. For example:
```
data_csv/
    ash_tree/
        ash_tree.csv
    bigleaf_maple/
        bigleaf_maple.csv
        ...
```
The images will be saved in a data directory nested under experiment name and class name:
```
data/
    trees/
        ash_tree/
            *.jpeg
```

In [5]:
for folder in os.listdir('../../data_csv'):
    file = path/folder/os.listdir(f'../../data_csv/{folder}')[0]
    dest = img_path/folder
    dest.mkdir(parents=True, exist_ok=True)
    print(folder, '   ',file, ' dest', dest)
    try:
        download_images(file, dest, max_pics=200, max_workers=0)
    except Exception as e:
        print(e,' failed to download image for: ', folder)
        pass

SystemExit: 1

## Verify Images can be Trained

In [ ]:
classes = os.listdir(f'../../data/{experiment}')
for c in classes:
    print(c)
    verify_images(img_path/c, delete=True, max_size=500)

## Create ImageDataBunch for Training

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(img_path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

## Show Classes to be Trained

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[error_rate, accuracy], callback_fns=[
    partial(callbacks.EarlyStoppingCallback, monitor='accuracy', min_delta=0.01, patience=5)])

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.save('stage-0')

In [ ]:
learn.load('stage-0')

In [ ]:
lr=1e-2

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(lr))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

## Save Model and Unfreeze Layers

In [ ]:
learn.save('stage-1')

In [ ]:
learn.load('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
starting_lr = 3e-4

## Train All Layers

In [ ]:
learn.fit_one_cycle(150, max_lr=slice(starting_lr,lr/5),callbacks=[ShowGraph(learn),callbacks.SaveModelCallback(learn,
monitor='error_rate',
mode='min')])

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

# Load Best Model, and Export for Production

In [ ]:
learn.load('bestmodel')

In [ ]:
learn.export('exported.pkl')